In [78]:
import tensorflow as tf
import keras

import matplotlib as plt
import numpy as np


In [93]:
from keras.applications.inception_v3 import InceptionV3,decode_predictions
from keras import backend as K

In [94]:
iv3 = InceptionV3()


In [81]:
print(iv3.summary())

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, 149, 149, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, 149, 149, 32) 96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 149, 149, 32) 0           batch_normalization_376[0][0]    
_______________________________________________________________________________________

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
bmw_path = "/content/drive/My Drive/data/images/bmw-serie-1.jpg"
mercedes_path = "/content/drive/My Drive/data/images/mercedes_clase_a.jpg"
airplane_path = "/content/drive/My Drive/data/images/airplane.jpg"

In [84]:
from keras.preprocessing import image

#to manipulate the images as a different representation we can use the function img_to_array 
x = image.img_to_array(image.load_img(bmw_path,target_size=(299,299)))  #inception need the image in size 299 x 299
a = image.img_to_array(image.load_img(airplane_path,target_size=(299,299)))  #inception need the image in size 299 x 299
#Range change, 0-255 -> -1 - 1

x /= 255
x -= 0.5
x *= 2

a /=255
a-=0.5
a*=2

x = x.reshape([1,x.shape[0],x.shape[1],x.shape[2]])

a = a.reshape([1,a.shape[0],a.shape[1],a.shape[2]])
print(x.shape)


(1, 299, 299, 3)


In [85]:
y = iv3.predict(x)
y2 = iv3.predict(a)

In [86]:
y.shape

(1, 1000)

In [87]:
decode_predictions(y)

[[('n02814533', 'beach_wagon', 0.36391553),
  ('n02974003', 'car_wheel', 0.22239973),
  ('n04285008', 'sports_car', 0.120032415),
  ('n03459775', 'grille', 0.11253346),
  ('n04037443', 'racer', 0.018364294)]]

In [88]:
decode_predictions(y2)

[[('n02690373', 'airliner', 0.88838637),
  ('n04592741', 'wing', 0.027158279),
  ('n02692877', 'airship', 0.0021623436),
  ('n04467665', 'trailer_truck', 0.00078004657),
  ('n02483362', 'gibbon', 0.00062221405)]]

<h1>Adversarial Attack<h1>

In [ ]:
inp_layer = iv3.layers[0].input

out_layer = iv3.layers[-1].output

print(inp_layer)
print(out_layer)

target_class = 951 #this number corresponds to a lemmon 
loss = out_layer[0,target_class]

grad = K.gradients(loss,inp_layer)[0]

optimize_gredient = K.function([inp_layer,K.learning_phase()],[grad,loss])
adv= np.copy(x)

pert = 0.01

max_pert = x + pert
min_pert = x - pert

cost = 0.0

while cost < 0.95:
  gr, cost = optimize_gredient([adv,0])
  adv+=gr
  adv = np.clip(adv,min_pert,max_pert)
  adv= np.clip(adv,-1,1)
  print("lemon Cost: ",cost)

hacked_img = np.copy(adv)

In [ ]:
adv /= 2
adv +=0.5
adv*=255

plt.imshow(adv[0],astype(np.uint8))
plt.show()